In [4]:
#-*- coding=utf-8 -*-
import requests
import json
import time
import hashlib
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import datetime
import os, sys, errno
import threading
%matplotlib inline


def check_file_path_exist(csvName):
    if not os.path.exists(os.path.dirname(csvName)):
        try:
            os.makedirs(os.path.dirname(csvName))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
                
def get_datetime_from_timeString(date_string):
    date_string = date_string.replace('T',' ').replace('Z','')
    datetime_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')
    return datetime_object

def get_two_datetime_delta(datetime_object1 , datetime_object2):
    datetime_delta   = datetime_object2 - datetime_object1
    return datetime_delta.total_seconds()

In [1]:
def byteToMagData(Low_byte, High_byte):
    value = int(Low_byte.encode(), 16) + int(High_byte.encode(), 16) * 256
    return twos_comp(value, 16)

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val 

def getPraseTargetData(Data):
    prase_data = Data['value']['hits']
    prase_data_from_json = []
    latency_time_list = list()
    create_time_list  = list()
    # 'Asia/Taipei' LMT+8:06:00 STD
    TPE_UTC_offset = 60*60*8 
    
    # columes in data frame
    prase_data_columes = ['_id', '_index', 'mac','create_at_time','recv_at_time','frameCnt', 'data',
                          'company','road','slot_num','gwip','rssi','snr',
                          "timestamp","latency_time","magX", "magY", "magZ","mag_len" ,"magT"]

    # get data by list
    for each in prase_data:
        fullData = []
        pakcetHeader = "fcffff07"
        packetByte = []
        
        fullData = [ each['_id'], each['_index'], each['_source']['macAddr'],
             each['_source']['create_at'], each['_source']['recv'], each['_source']['frameCnt'],
             each['_source']['data'], 
             each['_source']['extra']['company'],  each['_source']['extra']['road'], 
             each['_source']['extra']['slot_num'], each['_source']['gwip'],
             # each['_source']['dataParse']['voltage'], each['_source']['dataParse']['lowBattery'],
             each['_source']['rssi'], each['_source']['snr']
        ]
        
        create_time = get_datetime_from_timeString( each['_source']['create_at'])
        recv_time   = get_datetime_from_timeString( each['_source']['recv'] )
        latency_time = get_two_datetime_delta( recv_time , create_time )
        
        # get high/ low byte
        packetMagData = each['_source']['data']
        
        index = [i for i in range(len(packetMagData)) if i % 2 == 0]   
        for each in index:
            packetByte.append(packetMagData[each:each+2])
        
#         print("packet Byte: ", packetByte)
        
        magX = byteToMagData(packetByte[-7], packetByte[-6])
        magY = byteToMagData(packetByte[-5], packetByte[-4])
        magZ = byteToMagData(packetByte[-3], packetByte[-2])
        magT = byteToMagData(packetByte[-1], '00')
        maglen = (magX**2 + magY**2 + magZ**2)**(0.5)

        fullData.extend([recv_time.timestamp() + TPE_UTC_offset, latency_time, magX, magY, magZ, maglen, magT])
#         print("Data:", fullData)
        prase_data_from_json.append(fullData)
    return pd.DataFrame(prase_data_from_json, columns=prase_data_columes) 


In [2]:
def fast_plot_line(figure, axis, index, time , x, y, z, length ,title, ylimit):
    indexRow = int(index%5)
    indexCol = int(index/5)
    axis[indexRow][indexCol].plot()
    axis[indexRow][indexCol].plot(time, x, label='magX')
    axis[indexRow][indexCol].plot(time, y, label='magY')
    axis[indexRow][indexCol].plot(time, z, label='magZ')
    axis[indexRow][indexCol].plot(time, length, label='magLen')
#     axis[index][0].set_xlabel('Time')
    axis[indexRow][indexCol].set_ylabel('Mag Value')
    axis[indexRow][indexCol].set_title(title)
    axis[indexRow][indexCol].legend()
    axis[indexRow][indexCol].grid()
    if ylimit:
        axis[index].set_ylim([-100, 100])
    return None

def plot_basic_plot(count, magX, magY, magZ, title):
    plt.figure()
    plt.plot(count, magX, label='magX')
    plt.plot(count, magY, label='magY')
    plt.plot(count, magZ, label='magZ')
    plt.xlabel('Time')
    plt.ylabel('Mag Value')
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

def get_secific_time_data(data, ex_start_time, ex_end_time , offset = 0):
    data = data.loc[data['timestamp'] >= ex_start_time ]
    data = data.loc[data['timestamp'] <= ex_end_time]
    data = data.drop_duplicates(['frameCnt'])
    data = data.sort_values(['frameCnt'])
    return data

def get_data_lose_rate(node_name, expected_points, actual_points):
    lose_Point = int(expected_points - actual_points)
    lose_Rate = lose_Point / query_data_points * 100
    print("Node:", node_name, " - data pts in this period:", dfenvlen, 
          ", data lose pts:", lose_Point, ", lose rate:", lose_Rate )
    
def get_mag_offset_data(data, EnvMagX, EnvMagY, EnvMagZ, offset = 60*60*8):

    data["count"] = pd.to_datetime((data['timestamp'] + offset).astype(int),unit='s')
    data["magX_of"] = data["magX"] - EnvMagX
    data["magY_of"] = data["magY"] - EnvMagY
    data["magZ_of"] = data["magZ"] - EnvMagZ
    data["magLen_of"] = (data["magX_of"].pow(2) + data["magY_of"].pow(2) + data["magZ_of"].pow(2) ).pow(0.5)
    return data

## 2. plot data

In [40]:
folderName = os.getcwd() + '/0822_MatrixALL/pi3_data/'
# experiemtName = folderName + 'Veryca2_Right/'
# experiemtName = folderName + 'Veryca3_Back/'
experiemtName = folderName + 'Veryca4_Hold/'

csvFiles = [f for f in listdir(experiemtName) if isfile(join(experiemtName, f)) and f[-3:] == "csv" ]


header = ["time", "Mag1_X","Mag1_Y","Mag1_Z","Mag2_X","Mag2_Y","Mag2_Z",
          "Mag3_X","Mag3_Y","Mag3_Z","Mag4_X","Mag4_Y","Mag4_Z"]

df1_prase = pd.DataFrame()
df2_prase = pd.DataFrame()
# df = pd.DataFrame()

def get_mag_len(df):
    df["Mag1_len"] = (df["Mag1_X"].pow(2) + df["Mag1_Y"].pow(2) + df["Mag1_Z"].pow(2) ).pow(0.5)
    df["Mag2_len"] = (df["Mag2_X"].pow(2) + df["Mag2_Y"].pow(2) + df["Mag2_Z"].pow(2) ).pow(0.5)
    df["Mag3_len"] = (df["Mag3_X"].pow(2) + df["Mag3_Y"].pow(2) + df["Mag3_Z"].pow(2) ).pow(0.5)
    df["Mag4_len"] = (df["Mag4_X"].pow(2) + df["Mag4_Y"].pow(2) + df["Mag4_Z"].pow(2) ).pow(0.5)
    return df

for csv in csvFiles:
    df1 = pd.read_csv(experiemtName + csv, names = header,  header=None)
    df1 = get_mag_len(df1)
#     df1.to_csv(experiemtName + 'revised_' + csv)
    df1_log = df1.iloc[:,1:].mean()
    
    key = csv.strip(".csv");
    df1_prase[key] = df1_log
    df2_prase[key] = df2_log
    if csv[:3] == "env":
        savecsv = csv

df2 = df1.iloc[:,1:] - df1_log
df2 = get_mag_len(df2)
df2_log = df2.mean()     
df2_prase[key] = df2_log
    
    
saveName1 = folderName + "statistic_" + savecsv
saveName2 = folderName + "statistic_offset_" + savecsv
# df_prase = df_prase.transpose()
df1_prase = df1_prase[sorted(df_prase.columns)]
df1_prase.to_csv(saveName1)

df2_prase = df2_prase[sorted(df_prase.columns)]
df2_prase.to_csv(saveName2)

In [41]:
df2

,Mag1_X,Mag1_Y,Mag1_Z,Mag2_X,Mag2_Y,Mag2_Z,Mag3_X,Mag3_Y,Mag3_Z,Mag4_X,Mag4_Y,Mag4_Z,Mag1_len,Mag2_len,Mag3_len,Mag4_len
0,2.144144,-0.855856,1.945946,-3.540541,-2.774775,-0.072072,-1.135135,-2.567568,-3.513514,-0.198198,-1.342342,0.774775,3.019362,4.498888,4.497301,1.562511
1,3.144144,-2.855856,0.945946,-1.540541,-0.774775,6.927928,-4.135135,-1.567568,-3.513514,0.801802,1.657658,-0.225225,4.351594,7.139309,5.648131,1.855112
2,2.144144,-0.855856,-1.054054,0.459459,-2.774775,3.927928,-3.135135,-1.567568,-2.513514,0.801802,0.657658,0.774775,2.537888,4.831055,4.313246,1.294479
3,-0.855856,-0.855856,-5.054054,2.459459,-1.774775,2.927928,-2.135135,-1.567568,0.486486,0.801802,0.657658,1.774775,5.196965,4.215629,2.693091,2.055535
4,-0.855856,-0.855856,-4.054054,3.459459,-0.774775,-0.072072,-1.135135,-1.567568,2.486486,-0.198198,-0.342342,-0.225225,4.230878,3.545889,3.150939,0.455200
5,-2.855856,-2.855856,-4.054054,1.459459,-0.774775,-3.072072,-0.135135,-1.567568,1.486486,-0.198198,-0.342342,2.774775,5.722515,3.488255,2.164526,2.802830
6,-2.855856,0.144144,-1.054054,0.459459,-1.774775,-6.072072,2.864865,-1.567568,2.486486,-0.198198,-0.342342,1.774775,3.047576,6.342790,4.104550,1.818325
7,-2.855856,-1.855856,0.945946,-1.540541,-1.774775,-5.072072,3.864865,-1.567568,3.486486,-0.198198,1.657658,-4.225225,3.534816,5.590081,5.435995,4.543087
8,-0.855856,-1.855856,3.945946,-2.540541,-1.774775,-4.072072,1.864865,-0.567568,0.486486,-0.198198,0.657658,1.774775,4.443780,5.117220,2.009110,1.903056
9,1.144144,-2.855856,0.945946,-2.540541,-2.774775,-0.072072,-0.135135,-1.567568,-2.513514,-1.198198,1.657658,-5.225225,3.218663,3.762833,2.965346,5.611282


In [25]:
dflog

Mag1_X      200.855856
Mag1_Y      302.855856
Mag1_Z     -312.945946
Mag2_X      150.540541
Mag2_Y       54.774775
Mag2_Z     -357.927928
Mag3_X      266.135135
Mag3_Y      117.567568
Mag3_Z     -262.486486
Mag4_X       -0.801802
Mag4_Y       -0.657658
Mag4_Z       -0.774775
Mag1_len    479.598916
Mag2_len    392.157643
Mag3_len    391.874527
Mag4_len      2.369873
dtype: float64

In [21]:
for colume in df_prase.columns
    if colume[:3] not "env":
        

'back_hold_veryca'

In [22]:
df_prase.columns

Index(['back_hold_veryca', 'env_hold', 'forward_hold_RV'], dtype='object')